In [ ]:
%matplotlib inline

from copy import deepcopy
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 1) Chargement et analyse de la machine de référence
ref_path = join(DATA_DIR, "Machine", "Toyota_Prius.json")
M_ref = load(ref_path)

print("=" * 60)
print("ANALYSE DE LA MACHINE TOYOTA PRUIS DE RÉFÉRENCE")
print("=" * 60)

# === ANALYSE GÉNÉRALE ===
print(f"\n INFORMATIONS GÉNÉRALES:")
print(f"   Type de machine: {type(M_ref).__name__}")
print(f"   Nom: {getattr(M_ref, 'name', 'Non défini')}")

# === ANALYSE DU STATOR ===
print(f"\n STATOR:")
print(f"   Rayon interne: {M_ref.stator.Rint*1000:.2f} mm")
print(f"   Rayon externe: {M_ref.stator.Rext*1000:.2f} mm")
print(f"   Longueur active: {M_ref.stator.L1*1000:.2f} mm")
print(f"   Nombre d'encoches: {M_ref.stator.slot.Zs}")

# Paramètres d'encoche
slot = M_ref.stator.slot
print(f"\n  ENCOCHES STATOR:")
print(f"   Type d'encoche: {type(slot).__name__}")
if hasattr(slot, 'W0'): print(f"   Largeur ouverture (W0): {slot.W0*1000:.2f} mm")
if hasattr(slot, 'W1'): print(f"   Largeur sommet (W1): {slot.W1*1000:.2f} mm")
if hasattr(slot, 'W2'): print(f"   Largeur base (W2): {slot.W2*1000:.2f} mm")
if hasattr(slot, 'H0'): print(f"   Hauteur isthme (H0): {slot.H0*1000:.2f} mm")
if hasattr(slot, 'H1'): print(f"   Hauteur coin (H1): {slot.H1*1000:.2f} mm")
if hasattr(slot, 'H2'): print(f"   Hauteur encoche (H2): {slot.H2*1000:.2f} mm")

# Bobinage
winding = M_ref.stator.winding
print(f"\n  BOBINAGE:")
print(f"   Nombre de phases: {winding.qs}")
print(f"   Nombre de paires de pôles: {winding.p}")
print(f"   Nombre de couches: {winding.Nlayer}")
print(f"   Tours par bobine: {winding.Ntcoil}")
print(f"   Pas de bobine: {winding.coil_pitch}")

# === ANALYSE DU ROTOR ===
print(f"\ ROTOR:")
print(f"   Rayon interne: {M_ref.rotor.Rint*1000:.2f} mm")
print(f"   Rayon externe: {M_ref.rotor.Rext*1000:.2f} mm")
print(f"   Longueur active: {M_ref.rotor.L1*1000:.2f} mm")

# Entrefer
airgap = M_ref.stator.Rint - M_ref.rotor.Rext
print(f"\n   ENTREFER:")
print(f"   Valeur: {airgap*1000:.2f} mm")
print(f"   Pourcentage du rayon: {(airgap/M_ref.stator.Rint)*100:.2f}%")

# === ANALYSE DES AIMANTS ===
print(f"\n   AIMANTS:")
print(f"   Type de rotor: {type(M_ref.rotor).__name__}")

# Accès aux aimants via les trous du rotor
magnet_list = []
if hasattr(M_ref.rotor, 'hole') and M_ref.rotor.hole:
    print(f"   Nombre de trous: {len(M_ref.rotor.hole)}")
    
    for i, hole in enumerate(M_ref.rotor.hole):
        print(f"\n    TROU {i+1}:")
        print(f"     Type: {type(hole).__name__}")
        if hasattr(hole, 'Zh'): print(f"     Nombre autour: {hole.Zh}")
        if hasattr(hole, 'W0'): print(f"     Largeur ouverture: {hole.W0*1000:.2f} mm")
        if hasattr(hole, 'H0'): print(f"     Profondeur: {hole.H0*1000:.2f} mm")
        
        # Recherche des aimants dans ce trou
        hole_magnets = []
        for attr_name in dir(hole):
            if attr_name.startswith('magnet_'):
                magnet = getattr(hole, attr_name)
                if magnet is not None:
                    hole_magnets.append(magnet)
                    magnet_list.append(magnet)
        
        print(f"     Nombre d'aimants dans ce trou: {len(hole_magnets)}")
        
        for j, magnet in enumerate(hole_magnets):
            print(f"     Aimant {j+1}:")
            if hasattr(magnet, 'Hmag'): 
                print(f"       Épaisseur: {magnet.Hmag*1000:.2f} mm")
            if hasattr(magnet, 'Wmag'): 
                print(f"       Largeur: {magnet.Wmag*1000:.2f} mm")
            if hasattr(magnet, 'mat_type'): 
                print(f"       Matériau: {magnet.mat_type.name}")
                if hasattr(magnet.mat_type, 'mag') and hasattr(magnet.mat_type.mag, 'Brm20'):
                    print(f"       Br (20°C): {magnet.mat_type.mag.Brm20:.2f} T")
            if hasattr(magnet, 'type_magnetization'):
                print(f"       Type magnétisation: {magnet.type_magnetization}")

print(f"\n   RÉSUMÉ AIMANTS:")
print(f"   Total d'aimants: {len(magnet_list)}")

# === CALCULS DE COHÉRENCE ===
print(f"\n VÉRIFICATIONS DE COHÉRENCE:")

# Vérification entrefer
if airgap > 0:
    print(f"    Entrefer positif: {airgap*1000:.2f} mm")
    if airgap < 1e-3:  # 1 mm
        print(f"  Entrefer très petit (< 1mm)")
    elif airgap > 3e-3:  # 3 mm
        print(f"  Entrefer très grand (> 3mm)")
    else:
        print(f"  Entrefer dans la plage normale")
else:
    print(f"  Entrefer négatif: {airgap*1000:.2f} mm")

# Vérification dimensions aimants
if magnet_list:
    for i, magnet in enumerate(magnet_list):
        if hasattr(magnet, 'Hmag') and hasattr(magnet, 'Wmag'):
            ratio = magnet.Hmag / magnet.Wmag if magnet.Wmag > 0 else 0
            print(f"   Aimant {i+1} - Ratio épaisseur/largeur: {ratio:.2f}")
            if ratio > 0.1 and ratio < 2.0:
                print(f"     Ratio raisonnable")
            else:
                print(f"     Ratio à vérifier")

# Vérification bobinage
if winding.Ntcoil > 0:
    print(f"   Nombre de tours valide: {winding.Ntcoil}")
else:
    print(f"    Nombre de tours invalide: {winding.Ntcoil}")

# Vérification géométrie générale
stator_thickness = M_ref.stator.Rext - M_ref.stator.Rint
rotor_thickness = M_ref.rotor.Rext - M_ref.rotor.Rint
print(f"\n   GÉOMÉTRIE:")
print(f"   Épaisseur stator: {stator_thickness*1000:.2f} mm")
print(f"   Épaisseur rotor: {rotor_thickness*1000:.2f} mm")
print(f"   Ratio stator/rotor: {stator_thickness/rotor_thickness:.2f}")

print("\n" + "=" * 60)